## Preprocessing

In [24]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

EIN                                      NAME APPLICATION_TYPE  \
0  10520599              BLUE KNIGHTS MOTORCYCLE CLUB              T10   
1  10531628    AMERICAN CHESAPEAKE CLUB CHARITABLE TR               T3   
2  10547893        ST CLOUD PROFESSIONAL FIREFIGHTERS               T5   
3  10553066            SOUTHSIDE ATHLETIC ASSOCIATION               T3   
4  10556103  GENETIC RESEARCH INSTITUTE OF THE DESERT               T3   

        AFFILIATION CLASSIFICATION      USE_CASE  ORGANIZATION  STATUS  \
0       Independent          C1000    ProductDev   Association       1   
1       Independent          C2000  Preservation  Co-operative       1   
2  CompanySponsored          C3000    ProductDev   Association       1   
3  CompanySponsored          C2000  Preservation         Trust       1   
4       Independent          C1000     Heathcare         Trust       1   

      INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  IS_SUCCESSFUL  
0              0                      N     5000              1  
1         1-9999                      N   108590              1  
2              0                      N     5000              0  
3    10000-24999                      N     6692              1  
4  100000-499999                      N   142590              1

In [25]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
filtered_df = application_df.drop(columns=["NAME", "EIN"])
filtered_df.head()

APPLICATION_TYPE       AFFILIATION CLASSIFICATION      USE_CASE  \
0              T10       Independent          C1000    ProductDev   
1               T3       Independent          C2000  Preservation   
2               T5  CompanySponsored          C3000    ProductDev   
3               T3  CompanySponsored          C2000  Preservation   
4               T3       Independent          C1000     Heathcare   

   ORGANIZATION  STATUS     INCOME_AMT SPECIAL_CONSIDERATIONS  ASK_AMT  \
0   Association       1              0                      N     5000   
1  Co-operative       1         1-9999                      N   108590   
2   Association       1              0                      N     5000   
3         Trust       1    10000-24999                      N     6692   
4         Trust       1  100000-499999                      N   142590   

   IS_SUCCESSFUL  
0              1  
1              1  
2              0  
3              1  
4              1

In [26]:
# Determine the number of unique values in each column.
filtered_cat = filtered_df.dtypes[filtered_df.dtypes == "object"].index.tolist()
filtered_df[filtered_cat].nunique()

APPLICATION_TYPE          17
AFFILIATION                6
CLASSIFICATION            71
USE_CASE                   5
ORGANIZATION               4
INCOME_AMT                 9
SPECIAL_CONSIDERATIONS     2
dtype: int64

In [27]:
# Look at APPLICATION_TYPE value counts for binning
app_type = filtered_df['APPLICATION_TYPE'].value_counts().reset_index()
app_type

index  APPLICATION_TYPE
0     T3             27037
1     T4              1542
2     T6              1216
3     T5              1173
4    T19              1065
5     T8               737
6     T7               725
7    T10               528
8     T9               156
9    T13                66
10   T12                27
11    T2                16
12   T25                 3
13   T14                 3
14   T29                 2
15   T15                 2
16   T17                 1

In [28]:
# Choose a cutoff value and create a list of application types to be replaced
application_types_to_replace = []
#T9 156
application_types_to_replace = app_type[app_type['APPLICATION_TYPE'] <= 156]['index']

# Replace in dataframe
for app in application_types_to_replace:
    filtered_df['APPLICATION_TYPE'] = filtered_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
filtered_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [29]:
# Look at CLASSIFICATION value counts for binning
class_count = filtered_df['CLASSIFICATION'].value_counts().reset_index()
class_count

index  CLASSIFICATION
0   C1000           17326
1   C2000            6074
2   C1200            4837
3   C3000            1918
4   C2100            1883
..    ...             ...
66  C4120               1
67  C8210               1
68  C2561               1
69  C4500               1
70  C2150               1

[71 rows x 2 columns]

In [30]:
# You may find it helpful to look at CLASSIFICATION value counts >1
class_count_filtered = []
class_count_filtered = class_count[class_count['CLASSIFICATION'] > 1]
class_count_filtered

index  CLASSIFICATION
0   C1000           17326
1   C2000            6074
2   C1200            4837
3   C3000            1918
4   C2100            1883
5   C7000             777
6   C1700             287
7   C4000             194
8   C5000             116
9   C1270             114
10  C2700             104
11  C2800              95
12  C7100              75
13  C1300              58
14  C1280              50
15  C1230              36
16  C1400              34
17  C7200              32
18  C2300              32
19  C1240              30
20  C8000              20
21  C7120              18
22  C1500              16
23  C1800              15
24  C6000              15
25  C1250              14
26  C8200              11
27  C1238              10
28  C1278              10
29  C1235               9
30  C1237               9
31  C7210               7
32  C2400               6
33  C1720               6
34  C4100               6
35  C1257               5
36  C1600               5
37  C1260               3
38  C2710               3
39     C0               3
40  C3200               2
41  C1234               2
42  C1246               2
43  C1267               2
44  C1256               2

In [31]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`
classifications_to_replace = []
#C2100-1883
classifications_to_replace = class_count[class_count['CLASSIFICATION'] < 1883]['index']

# Replace in dataframe
for cls in classifications_to_replace:
    filtered_df['CLASSIFICATION'] = filtered_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
filtered_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [32]:
# Convert categorical data to numeric with `pd.get_dummies`
numerical_df = pd.get_dummies(filtered_df)
numerical_df

STATUS   ASK_AMT  IS_SUCCESSFUL  APPLICATION_TYPE_Other  \
0           1      5000              1                       0   
1           1    108590              1                       0   
2           1      5000              0                       0   
3           1      6692              1                       0   
4           1    142590              1                       0   
...       ...       ...            ...                     ...   
34294       1      5000              0                       0   
34295       1      5000              0                       0   
34296       1      5000              0                       0   
34297       1      5000              1                       0   
34298       1  36500179              0                       0   

       APPLICATION_TYPE_T10  APPLICATION_TYPE_T19  APPLICATION_TYPE_T3  \
0                         1                     0                    0   
1                         0                     0                    1   
2                         0                     0                    0   
3                         0                     0                    1   
4                         0                     0                    1   
...                     ...                   ...                  ...   
34294                     0                     0                    0   
34295                     0                     0                    0   
34296                     0                     0                    1   
34297                     0                     0                    0   
34298                     0                     0                    1   

       APPLICATION_TYPE_T4  APPLICATION_TYPE_T5  APPLICATION_TYPE_T6  ...  \
0                        0                    0                    0  ...   
1                        0                    0                    0  ...   
2                        0                    1                    0  ...   
3                        0                    0                    0  ...   
4                        0                    0                    0  ...   
...                    ...                  ...                  ...  ...   
34294                    1                    0                    0  ...   
34295                    1                    0                    0  ...   
34296                    0                    0                    0  ...   
34297                    0                    1                    0  ...   
34298                    0                    0                    0  ...   

       INCOME_AMT_1-9999  INCOME_AMT_10000-24999  INCOME_AMT_100000-499999  \
0                      0                       0                         0   
1                      1                       0                         0   
2                      0                       0                         0   
3                      0                       1                         0   
4                      0                       0                         1   
...                  ...                     ...                       ...   
34294                  0                       0                         0   
34295                  0                       0                         0   
34296                  0                       0                         0   
34297                  0                       0                         0   
34298                  0                       0                         0   

       INCOME_AMT_10M-50M  INCOME_AMT_1M-5M  INCOME_AMT_25000-99999  \
0                       0                 0                       0   
1                       0                 0                       0   
2                       0                 0                       0   
3                       0                 0                       0   
4                       0                 0                       0   
...                   ...            

In [33]:
# Split our preprocessed data into our features and target arrays
y = numerical_df["IS_SUCCESSFUL"].values
X = numerical_df.drop(["IS_SUCCESSFUL"],1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

C:\Users\zjoni\AppData\Local\Temp\ipykernel_2328\120369778.py:3: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  X = numerical_df.drop(["IS_SUCCESSFUL"],1).values


In [34]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [35]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  120
hidden_nodes_layer2 = 90

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="LeakyReLU")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="LeakyReLU"))


# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_3 (Dense)             (None, 120)               5280      
                                                                 
 dense_4 (Dense)             (None, 90)                10890     
                                                                 
 dense_5 (Dense)             (None, 1)                 91        
                                                                 
Total params: 16261 (63.52 KB)
Trainable params: 16261 (63.52 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [36]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [37]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=100)

Epoch 1/100
804/804 [==============================] - 11s 8ms/step - loss: 0.5781 - accuracy: 0.7180
Epoch 2/100
804/804 [==============================] - 8s 9ms/step - loss: 0.5618 - accuracy: 0.7262
Epoch 3/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5583 - accuracy: 0.7292
Epoch 4/100
804/804 [==============================] - 7s 9ms/step - loss: 0.5554 - accuracy: 0.7276
Epoch 5/100
804/804 [==============================] - 9s 12ms/step - loss: 0.5535 - accuracy: 0.7296
Epoch 6/100
804/804 [==============================] - 6s 8ms/step - loss: 0.5530 - accuracy: 0.7287
Epoch 7/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5523 - accuracy: 0.7301
Epoch 8/100
804/804 [==============================] - 9s 11ms/step - loss: 0.5522 - accuracy: 0.7308
Epoch 9/100
804/804 [==============================] - 7s 8ms/step - loss: 0.5508 - accuracy: 0.7315
Epoch 10/100
804/804 [==============================] - 8s 10ms/step - loss: 0.5507 - ac

In [41]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 1s - loss: 0.5522 - accuracy: 0.7317 - 1s/epoch - 4ms/step
Loss: 0.5522150993347168, Accuracy: 0.7316617965698242


In [42]:
# Exporting the model to HDF5 file
nn.save('AlphabetSoupCharity.h5')
